In [63]:
!pip install pymongo
import pymongo
from pymongo import MongoClient
from pprint import pprint
import re
import pandas as pd
from datetime import datetime

In [64]:
client = MongoClient()
db = client.DEP

In [65]:
print("Q1: Return the total number of customers living in California")
ca_customers_count = db.customers.count_documents({"District": "California"})
pprint(f"Total Number of Customers in California: {ca_customers_count}")

Q1: Return the total number of customers living in California
'Total Number of Customers in California: 9'


In [66]:
print("Q2: List all movies that are rated NC-17. Return the title and the rating, sorted alphabetically by title.")
nc17_movies = db.films.find({"Rating": "NC-17"}, {"Title": 1, "Rating": 1}).sort("Title", pymongo.ASCENDING)
pprint(list(nc17_movies))

Q2: List all movies that are rated NC-17. Return the title and the rating, sorted alphabetically by title.
[{'Rating': 'NC-17', 'Title': 'ADAPTATION HOLES', '_id': 3},
 {'Rating': 'NC-17', 'Title': 'ALADDIN CALENDAR', '_id': 10},
 {'Rating': 'NC-17', 'Title': 'ALICE FANTASIA', '_id': 14},
 {'Rating': 'NC-17', 'Title': 'ALIEN CENTER', '_id': 15},
 {'Rating': 'NC-17', 'Title': 'ALLEY EVOLUTION', '_id': 16},
 {'Rating': 'NC-17', 'Title': 'ANONYMOUS HUMAN', '_id': 27},
 {'Rating': 'NC-17', 'Title': 'ANTITRUST TOMATOES', '_id': 29},
 {'Rating': 'NC-17', 'Title': 'APACHE DIVINE', '_id': 31},
 {'Rating': 'NC-17', 'Title': 'ARABIA DOGMA', '_id': 34},
 {'Rating': 'NC-17', 'Title': 'ARK RIDGEMONT', '_id': 38},
 {'Rating': 'NC-17', 'Title': 'ARTIST COLDBLOODED', '_id': 42},
 {'Rating': 'NC-17', 'Title': 'BABY HALL', '_id': 47},
 {'Rating': 'NC-17', 'Title': 'BALLOON HOMEWARD', '_id': 51},
 {'Rating': 'NC-17', 'Title': 'BANG KWAI', '_id': 53},
 {'Rating': 'NC-17', 'Title': 'BED HIGHBALL', '_id': 6

In [67]:
print("Q3: List the movie titles and the count of actors in each movie, sorted alphabetically by title.")
aggregation=[
    {"$project":{"Title":1,"Number of Actors":{"$size":"$Actors"}}},
    {"$sort":{"Title":1}},
]
actor_count_in_each_movie=db.films.aggregate(aggregation)
pprint(list(actor_count_in_each_movie))

Q3: List the movie titles and the count of actors in each movie, sorted alphabetically by title.
[{'Number of Actors': 10, 'Title': 'ACADEMY DINOSAUR', '_id': 1},
 {'Number of Actors': 4, 'Title': 'ACE GOLDFINGER', '_id': 2},
 {'Number of Actors': 5, 'Title': 'ADAPTATION HOLES', '_id': 3},
 {'Number of Actors': 5, 'Title': 'AFFAIR PREJUDICE', '_id': 4},
 {'Number of Actors': 5, 'Title': 'AFRICAN EGG', '_id': 5},
 {'Number of Actors': 7, 'Title': 'AGENT TRUMAN', '_id': 6},
 {'Number of Actors': 5, 'Title': 'AIRPLANE SIERRA', '_id': 7},
 {'Number of Actors': 4, 'Title': 'AIRPORT POLLOCK', '_id': 8},
 {'Number of Actors': 9, 'Title': 'ALABAMA DEVIL', '_id': 9},
 {'Number of Actors': 8, 'Title': 'ALADDIN CALENDAR', '_id': 10},
 {'Number of Actors': 4, 'Title': 'ALAMO VIDEOTAPE', '_id': 11},
 {'Number of Actors': 7, 'Title': 'ALASKA PHANTOM', '_id': 12},
 {'Number of Actors': 5, 'Title': 'ALI FOREVER', '_id': 13},
 {'Number of Actors': 4, 'Title': 'ALICE FANTASIA', '_id': 14},
 {'Number of 

In [68]:
print("Q4: List the count of movies by category. "
      "Return the category and the count, sorted by the count per category descending.")
aggregation=[
    {"$group": {"_id": "$Category", "Count": {"$sum": 1}}},
    {"$project": {"Category": "$_id", "_id": 0, "Count": 1}},  
    {"$sort": {"Count": -1}},
]
Number_of_Category=db.films.aggregate(aggregation)
pprint(list(Number_of_Category))

Q4: List the count of movies by category. Return the category and the count, sorted by the count per category descending.
[{'Category': 'Sports', 'Count': 74},
 {'Category': 'Foreign', 'Count': 73},
 {'Category': 'Family', 'Count': 69},
 {'Category': 'Documentary', 'Count': 68},
 {'Category': 'Animation', 'Count': 66},
 {'Category': 'Action', 'Count': 64},
 {'Category': 'New', 'Count': 63},
 {'Category': 'Drama', 'Count': 62},
 {'Category': 'Sci-Fi', 'Count': 61},
 {'Category': 'Games', 'Count': 61},
 {'Category': 'Children', 'Count': 60},
 {'Category': 'Comedy', 'Count': 58},
 {'Category': 'Classics', 'Count': 57},
 {'Category': 'Travel', 'Count': 57},
 {'Category': 'Horror', 'Count': 56},
 {'Category': 'Music', 'Count': 51}]


In [69]:
print("Q5: List the top 2 movies with movie length greater than 25mins or which has commentaries as special features. Return the movie title, length, and special features.")
top_2_films = db.films.find(
    {"$or": [{"Length": {"$gt": 25}}, {"Special Features": {"$regex": "Commentaries"}}]},
    {"Title": 1, "Length": 1, "Special Features": 1}
).limit(2)
pprint(list(top_2_films))

Q5: List the top 2 movies with movie length greater than 25mins or which has commentaries as special features. Return the movie title, length, and special features.
[{'Length': '126',
  'Special Features': 'Commentaries,Behind the Scenes',
  'Title': 'ALAMO VIDEOTAPE',
  '_id': 11},
 {'Length': '136',
  'Special Features': 'Commentaries,Deleted Scenes',
  'Title': 'ALASKA PHANTOM',
  '_id': 12}]


In [70]:
print("Q6: List top 10 countries with the most number of customers.")
top_countries = db.customers.aggregate([
    {"$group": {"_id": "$Country", "Count": {"$sum": 1}}},
    {"$project": {"Country": "$_id", "_id": 0, "Count": 1}}, 
    {"$sort": {"Count": -1}},
    {"$limit": 10}
])
pprint(list(top_countries))

Q6: List top 10 countries with the most number of customers.
[{'Count': 60, 'Country': 'India'},
 {'Count': 53, 'Country': 'China'},
 {'Count': 36, 'Country': 'United States'},
 {'Count': 31, 'Country': 'Japan'},
 {'Count': 30, 'Country': 'Mexico'},
 {'Count': 28, 'Country': 'Russian Federation'},
 {'Count': 28, 'Country': 'Brazil'},
 {'Count': 20, 'Country': 'Philippines'},
 {'Count': 15, 'Country': 'Turkey'},
 {'Count': 14, 'Country': 'Indonesia'}]


In [71]:
print("Q7: List all replacement costs along with count for each replacement cost.")
replacement_costs = db.films.aggregate([
    {"$group": {"_id": "$Replacement Cost", "Count": {"$sum": 1}}},
    {"$project": {"Replacement Cost": "$_id", "_id": 0, "Count": 1}},
    {"$sort": {"Replacement Cost": 1}},
])
pprint(list(replacement_costs))

Q7: List all replacement costs along with count for each replacement cost.
[{'Count': 49, 'Replacement Cost': '10.99'},
 {'Count': 49, 'Replacement Cost': '11.99'},
 {'Count': 55, 'Replacement Cost': '12.99'},
 {'Count': 55, 'Replacement Cost': '13.99'},
 {'Count': 51, 'Replacement Cost': '14.99'},
 {'Count': 37, 'Replacement Cost': '15.99'},
 {'Count': 38, 'Replacement Cost': '16.99'},
 {'Count': 47, 'Replacement Cost': '17.99'},
 {'Count': 42, 'Replacement Cost': '18.99'},
 {'Count': 50, 'Replacement Cost': '19.99'},
 {'Count': 57, 'Replacement Cost': '20.99'},
 {'Count': 55, 'Replacement Cost': '21.99'},
 {'Count': 55, 'Replacement Cost': '22.99'},
 {'Count': 45, 'Replacement Cost': '23.99'},
 {'Count': 38, 'Replacement Cost': '24.99'},
 {'Count': 43, 'Replacement Cost': '25.99'},
 {'Count': 46, 'Replacement Cost': '26.99'},
 {'Count': 53, 'Replacement Cost': '27.99'},
 {'Count': 41, 'Replacement Cost': '28.99'},
 {'Count': 53, 'Replacement Cost': '29.99'},
 {'Count': 41, 'Replaceme

In [72]:
print("Q8: List all films relating to Mexico for a Cinco de Mayo promotion special.")
mexico_films = db.films.find(
    {"$or": [
        {"Description": {"$regex": "Mexico", "$options": "i"}},
        {"Title": {"$regex": "Mexico", "$options": "i"}}
    ]},
    {"Title": 1, "Description": 1}
)
pprint(list(mexico_films))

Q8: List all films relating to Mexico for a Cinco de Mayo promotion special.
[{'Description': 'A Fast-Paced Documentary of a Pastry Chef And a Dentist who '
                 'must Pursue a Forensic Psychologist in The Gulf of Mexico',
  'Title': 'AFRICAN EGG',
  '_id': 5},
 {'Description': 'A Epic Display of a Pioneer And a Student who must Challenge '
                 'a Butler in The Gulf of Mexico',
  'Title': 'BEETHOVEN EXORCIST',
  '_id': 64},
 {'Description': 'A Fateful Panorama of a Crocodile And a Boy who must Defeat '
                 'a Monkey in The Gulf of Mexico',
  'Title': 'BOONDOCK BALLROOM',
  '_id': 87},
 {'Description': 'A Lacklusture Character Study of a Husband And a Sumo '
                 'Wrestler who must Succumb a Technical Writer in The Gulf of '
                 'Mexico',
  'Title': 'BOOGIE AMELIE',
  '_id': 86},
 {'Description': 'A Brilliant Panorama of a Dentist And a Moose who must Find '
                 'a Student in The Gulf of Mexico',
  'Title': 'COL

In [73]:
#Question9 skip

In [74]:
print("Q10: Calculate revenue for the Month of August 2005.")
august_revenue= db.customers.aggregate([
    {"$unwind": "$Rentals"},
    {"$match": {
        "Rentals.Rental Date": {"$gte": "2005-08-01", "$lt": "2005-09-01"}
    }},
    {"$unwind": "$Rentals.Payments"},
    {"$group": {
        "_id": None,
        "Total Revenue": {"$sum": "$Rentals.Payments.Amount"}
    }},
    {"$project": {"Total Revenue": 1, "_id": 0}}  
])
august_revenue = list(august_revenue)
pprint(august_revenue[0])

Q10: Calculate revenue for the Month of August 2005.
{'Total Revenue': 24070.139395952225}


In [75]:
print("Q11: Find movies with the highest number of Actors.")
aggregation = [
    {"$project": {
        "Title": 1,
        "Actor Count": {"$size": "$Actors"}
    }},
    {"$sort": {"Actor Count": -1}},
    {"$limit": 1}
]
most_actor_movies = db.films.aggregate(aggregation)
pprint(list(most_actor_movies))

Q11: Find movies with the highest number of Actors.
[{'Actor Count': 15, 'Title': 'LAMBS CINCINATTI', '_id': 508}]


In [76]:
print("Q12: List top Movie rating that is rented the most / popular among the customers.")
top_rating = db.customers.aggregate([
    {"$unwind": "$Rentals"},
    {"$group": {"_id": "$Rentals.Film Title", "Count": {"$sum": 1}}},
    {"$lookup": {
        "from": "films",
        "localField": "_id",
        "foreignField": "Title",
        "as": "FilmData"
    }},
    {"$unwind": "$FilmData"},
    {"$group": {"_id": "$FilmData.Rating", "Total Rentals": {"$sum": "$Count"}}},
    {"$sort": {"Total Rentals": -1}},
    {"$limit": 1},
    {"$project": {"Rating": "$_id", "_id": 0, "Total Rentals": 1}}  
])
top_rating = list(top_rating)
pprint(top_rating[0]) 

Q12: List top Movie rating that is rented the most / popular among the customers.
{'Rating': 'PG-13', 'Total Rentals': 3585}


In [77]:
print("Q13: List number of actors who worked in each category of films.")
actor_category_counts = db.films.aggregate([
    {"$unwind": "$Actors"},
    {"$group": {"_id": {"Category": "$Category", "Actor": {"$concat": ["$Actors.Last name", ", ", "$Actors.First name"]}}}},
    {"$group": {"_id": "$_id.Category", "Unique Actors": {"$sum": 1}}},
    {"$sort": {"Unique Actors": -1}},
])

for category_count in actor_category_counts:
    pprint(category_count)

Q13: List number of actors who worked in each category of films.
{'Unique Actors': 181, '_id': 'Sports'}
{'Unique Actors': 174, '_id': 'Foreign'}
{'Unique Actors': 169, '_id': 'New'}
{'Unique Actors': 167, '_id': 'Documentary'}
{'Unique Actors': 166, '_id': 'Sci-Fi'}
{'Unique Actors': 166, '_id': 'Animation'}
{'Unique Actors': 165, '_id': 'Action'}
{'Unique Actors': 165, '_id': 'Travel'}
{'Unique Actors': 164, '_id': 'Family'}
{'Unique Actors': 162, '_id': 'Children'}
{'Unique Actors': 161, '_id': 'Drama'}
{'Unique Actors': 161, '_id': 'Classics'}
{'Unique Actors': 155, '_id': 'Horror'}
{'Unique Actors': 150, '_id': 'Games'}
{'Unique Actors': 147, '_id': 'Comedy'}
{'Unique Actors': 143, '_id': 'Music'}


In [78]:
print("Q14: Find the average length of movies in each category.")

average_length_category = db.films.aggregate([
    {"$match": {"Length": {"$exists": True, "$ne": None}}},  
    {"$addFields": {
        "Numeric Length": {"$convert": {
            "input": "$Length",
            "to": "double",
            "onError": None,
            "onNull": None
        }}
    }},
    {"$group": {
        "_id": "$Category",
        "Average Length": {"$avg": "$Numeric Length"}
    }},
    {"$project": {
        "Category": "$_id",
        "_id": 0,
        "Average Length": {"$round": ["$Average Length", 2]}
    }},
    {"$sort": {"Category": 1}},
])

for category in average_length_category:
    pprint(category)

Q14: Find the average length of movies in each category.
{'Average Length': 111.61, 'Category': 'Action'}
{'Average Length': 111.02, 'Category': 'Animation'}
{'Average Length': 109.8, 'Category': 'Children'}
{'Average Length': 111.67, 'Category': 'Classics'}
{'Average Length': 115.83, 'Category': 'Comedy'}
{'Average Length': 108.75, 'Category': 'Documentary'}
{'Average Length': 120.84, 'Category': 'Drama'}
{'Average Length': 114.78, 'Category': 'Family'}
{'Average Length': 121.7, 'Category': 'Foreign'}
{'Average Length': 127.84, 'Category': 'Games'}
{'Average Length': 112.48, 'Category': 'Horror'}
{'Average Length': 113.65, 'Category': 'Music'}
{'Average Length': 111.13, 'Category': 'New'}
{'Average Length': 108.2, 'Category': 'Sci-Fi'}
{'Average Length': 128.2, 'Category': 'Sports'}
{'Average Length': 113.32, 'Category': 'Travel'}


In [79]:
print("Q15: List the customers with at least 40 rentals.")
customers_40_rentals_query = db.customers.aggregate([
    {"$project": {
        "First Name": 1,
        "Last Name": 1,
        "Country": 1,
        "Rental Count": {"$size": "$Rentals"}
    }},
    {"$match": {"Rental Count": {"$gte": 40}}},
])

customers_40_rentals_df = pd.DataFrame(list(customers_40_rentals_query))

print("Customers with at Least 40 Rentals:")
display(customers_40_rentals_df)

Q15: List the customers with at least 40 rentals.
Customers with at Least 40 Rentals:


,_id,Country,First Name,Last Name,Rental Count
0,75,Taiwan,TAMMY,SANDERS,41
1,144,Belarus,CLARA,SHAW,42
2,148,Runion,ELEANOR,HUNT,46
3,197,China,SUE,PETERS,40
4,236,Philippines,MARCIA,DEAN,42
5,469,Spain,WESLEY,BULL,40
6,526,United States,KARL,SEAL,45
